# Prototype Natural Gas Storage Contract Pricing Model


This notebook builds a **prototype pricing model** for a natural gas storage contract.
It uses historical price data to evaluate storage cash flows.


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:

# Load natural gas price data
df = pd.read_csv('/mnt/data/Nat_Gas (1).csv')
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')
df.set_index('Date', inplace=True)
df.head()


In [ ]:

# Linear price interpolation function
def get_price(date):
    date = pd.to_datetime(date)
    return np.interp(
        date.toordinal(),
        df.index.map(pd.Timestamp.toordinal),
        df['Price'].values
    )


In [ ]:

def price_storage_contract(
    injection_dates,
    withdrawal_dates,
    injection_rate,
    withdrawal_rate,
    max_volume,
    storage_cost_per_unit
):
    """
    Prototype pricing model for gas storage contract
    Assumptions:
    - Zero interest rates
    - No transport delay
    """
    inventory = 0.0
    cashflow = 0.0

    # Injection phase
    for d in injection_dates:
        qty = min(injection_rate, max_volume - inventory)
        cashflow -= qty * get_price(d)
        inventory += qty

    # Storage costs (applied monthly)
    cashflow -= inventory * storage_cost_per_unit * len(injection_dates)

    # Withdrawal phase
    for d in withdrawal_dates:
        qty = min(withdrawal_rate, inventory)
        cashflow += qty * get_price(d)
        inventory -= qty

    return cashflow


In [ ]:

# Test the pricing function
injection_dates = pd.date_range('2023-04-30', periods=3, freq='M')
withdrawal_dates = pd.date_range('2023-11-30', periods=3, freq='M')

price_storage_contract(
    injection_dates=injection_dates,
    withdrawal_dates=withdrawal_dates,
    injection_rate=10000,
    withdrawal_rate=10000,
    max_volume=30000,
    storage_cost_per_unit=0.05
)
